In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from datetime import datetime
from pathlib import Path
import pandas as pd
import pandas_read_xml as pdx
from pandas_profiling import ProfileReport
from anateldb.parser import *
from anateldb.query import *

In [ ]:
SIGLAS = {"Acre" : "AC",
          "Alagoas" : "AL",
          "Amapá": "AP",
          "Amazonas":"AM",
          "Bahia":"BA",
          "Ceará":"CE",
          "Espírito Santo":"ES",
          "Goiás":"GO",
          "Maranhão":"MA",
          "Mato Grosso":"MT",
          "Mato Grosso do Sul":"MS",
          "Minas Gerais": "MG",
          "Pará": "PA",
          "Paraíba":"PB",
          "Paraná":"PR",
          "Pernambuco":"PE",
          "Piauí":"PI",
          "Rio de Janeiro":"RJ",
          "Rio Grande do Norte":"RN",
          "Rio Grande do Sul":"RS",
          "Rondônia":"RO",
          "Roraima":"RR",
          "Santa Catarina":"SC",
          "São Paulo":"SP",
          "Sergipe":"SE",
          "Tocantins":"TO",
          "Distrito Federal":"DF"}

In [ ]:
pb = read_plano_basico('D:\OneDrive - ANATEL\GR01FI3\BaseDados\plano_basico.zip')
pb.tail()

,Serviço,Status,Entidade,Fistel,Município,UF,Id,CNPJ,Frequência,Classe,Latitude,Longitude,Canal,País
45448,FM,FM-AT-01,SISTEMA TRANSRIO DE COMUNICACAO LTDA,01008010561,Rio de Janeiro,RJ,603e93f71f151,30913990000110,"105,1",E1,"-22,946944444444","-43,219444444444",286,BRA
45449,RTV,TV-AT-01,REDE DE COMUNICACAO CIDADE LTDA,50400718928,Ji-Paraná,RO,603e97b38612d,34459933000163,177,C,"-10,885277777778","-61,951666666667",7,BRA
45450,RTVD,TV-AT-01,FUNDACAO JOAO PAULO II,50409270024,Cuiabá,MT,603e9df677686,50016039000175,593,A,"-15,585278","-56,081944",34,BRA
45451,RTVD,TV-AT-01,TELEVISAO CACHOEIRA DO SUL LTDA,50416280250,Santo Antônio da Patrulha,RS,603eb8b8b773f,89784037000161,623,C,"-29,8175","-50,519722222222",39,BRA
45452,FM,FM-AT-01,BARRINHA COMUNICACOES LTDA,50406330921,Barrinha,SP,603ec1c290774,05009032000171,"95,5",C,"-21,193611111111","-48,163888888889",238,BRA


In [ ]:
# pb_profile = ProfileReport(pb, minimal=True, dark_mode=True, title='Perfil Plano Básico - Mosaico')
# pb_profile

In [ ]:
es = read_estações('D:\OneDrive - ANATEL\GR01FI3\BaseDados\estações.zip')
es.tail()

,Serviço,Status,Entidade,Fistel,Município,UF,Id,Número da Estação,CNPJ
28012,RTVD,TV-C2,M. V. L - COMMUNICARE TELECOMUNICACOES LTDA,50419656170,Parauapebas,PA,5f2068e65ace5,,12071310000186
28013,RTVD,TV-C1,MERCES COMUNICACOES LTDA,50419663118,Delmiro Gouveia,AL,5f218fcfb0d84,,11322505000199
28014,RTVD,TV-C1,FUNDACAO EDUCACIONAL E CULTURAL DE IPANEMA,50433856696,Sabará,MG,5f32c1c918e6b,,04608796000110
28015,FM,FM-C2,RADIO ITAPIRANGA LTDA,50433860456,Itapiranga,SC,5f68d432841a5,,84375872000124
28016,FM,FM-C1,EMISSORAS SUL BRASILEIRAS LTDA,50433937009,Horizontina,RS,5f8dcc96f23f9,,95818506000119


In [ ]:
# es_profile = ProfileReport(es, minimal=True, dark_mode=True, title='Perfil Estações - Mosaico')
# es_profile

In [ ]:
RIGHT = ['@id', '@fistel', '@SiglaServico', '@state']
LEFT = ['@id', '@Fistel', '@Servico', '@Status']
ON = ['Id', 'Fistel', 'Serviço', 'Status']
df = pb.merge(es, on='Id', how='outer') #.drop(RIGHT, axis=1)
df.shape

(49340, 22)

In [ ]:
COLS = ['Serviço', 'Status', 'Entidade', 'Fistel', 'Município', 'UF', 'CNPJ']
for col in COLS:
    col_x = col + '_x'
    col_y = col + '_y'
    if df[col_x].count() > df[col_y].count():
        a,b = col_x, col_y
    else:
        a,b = col_y, col_x
        
    df.loc[df[a].isna(), a] = df.loc[df[a].isna(), b]
    df.drop(b, axis=1, inplace=True)
    df.rename({a: a[:-2]}, axis=1, inplace=True)
df = df[df.Frequência.notna()].reset_index(drop=True)

In [ ]:
m = pd.read_excel('D:\OneDrive - ANATEL\GR01FI3\BaseDados\municípios.xlsx')

In [ ]:
df['Coordenadas_do_Município'] = False

In [ ]:
df['Latitude'] = df.Latitude.str.replace(',', '.')
df['Longitude'] = df.Longitude.str.replace(',', '.')
df['Frequência'] = df.Frequência.str.replace(',', '.')
for row in df[df.Latitude.isna()].itertuples():
    df.loc[row.Index, ['Latitude', 'Longitude']] = m.loc[(m.Município == row.Município) & (m.UF == row.UF), ['Latitude', 'Longitude']].values.flatten().tolist()    
    df.loc[row.Index, 'Coordenadas_do_Município'] = True
df['Latitude'] = df.Latitude.astype('float')
df['Longitude'] = df.Longitude.astype('float')
df['Frequência'] = df.Frequência.astype('float')

In [ ]:
df = df[df.País == 'BRA'].reset_index(drop=True)

In [ ]:
# from geopy.geocoders import Nominatim
# from geopy import *
# geolocator = Nominatim(user_agent='anateldb')

# for row in tqdm(df[df.Município.isna()].itertuples()):
#     loc = geolocator.reverse((row.Latitude,row.Longitude))
#     df.loc[row.Index, 'UF'] = SIGLAS[loc.raw.get('address').get('state')]
#     df.loc[row.Index, 'Município'] = loc.raw.get('address').get('town')
    
#     elif not loc:
#         df.loc[row.Index, 'UF'] = row.País
#         df.loc[row.Index, 'Município'] = row.País            
#     else:
#         if (address := loc.raw.get('address')):
#             if uf := address.get('state'):
#                 df.loc[row.Index, 'UF'] = uf
#             else:
#                 df.loc[row.Index, 'UF'] = row.País
#             if cidade := address.get('city'):
#                 df.loc[row.Index, 'Município'] = cidade
#             else:
#                 df.loc[row.Index, 'Município'] = row.País            

In [ ]:
# merge_profile = ProfileReport(df, minimal=True, dark_mode=True, title='Perfil Estações + Plano Básico - Mosaico')
# merge_profile

In [ ]:
df.to_feather('D:\OneDrive - ANATEL\GR01FI3\BaseDados\mosaico.fth')

In [ ]:
df = pd.read_feather('D:\OneDrive - ANATEL\GR01FI3\BaseDados\mosaico.fth')

In [ ]:
df.Serviço.unique()

array(['OM', 'TV', 'TVA', 'RTV', 'RTVD', 'GTVD', 'PBTVD', 'FM'],
      dtype=object)

In [ ]:
# profile = ProfileReport(df, minimal=True, dark_mode=True, title='Perfil Mosaico')
# profile

In [ ]:
radcom = read_radcom('D:\OneDrive - ANATEL\GR01FI3\BaseDados')

In [ ]:
radcom.head()

,Frequência,Fase,Situação,Unidade,Entidade,Fistel,Número da Estação,Município,UF,Latitude,Longitude,CNPJ
0,104.9,3,A,MHz,ACADEMIA CULTURAL DE SANTA HELENA - ACULT - ST...,50011685115,641168764,Santa Helena,PR,-24.861389,-54.334722,00104477000117
1,87.9,3,M,MHz,ASSOCIACAO DOS MORADORES E PRODUT. RURAIS DE A...,50012524409,682699349,Assunção,PB,-7.074444,-36.731111,00284576000128
2,87.9,3,H,MHz,ASSOCIACAO BONFIM ESPERANCA- ABESPE,50011824689,659028590,Bonfim,MG,-20.323611,-44.246944,00575697000129
3,104.9,3,B,MHz,ASSOCIACAO DOS TRABALHADORES DE GUIMARANIA (ATG),50011398132,631410937,Guimarânia,MG,-18.843611,-46.792778,00792795000118
4,87.9,3,M,MHz,FUNDACAO ASSISTENCIAL LAR DA PAZ - FALP,50011398990,631412301,Dores do Indaiá,MG,-19.466667,-45.600000,00794510000188


In [ ]:
radcom['Serviço'] = '231'
radcom['Status'] = 'RADCOM'
radcom['Classe'] = radcom.Fase.str.strip() + '-' + radcom.Situação.str.strip()
radcom['Entidade'] = radcom.Entidade.str.rstrip().str.lstrip()

In [ ]:
rd = df.append(radcom).sort_values("Frequência").reset_index(drop=True) ; rd.tail()

,Serviço,Status,Entidade,Município,UF,Id,Frequência,Classe,Latitude,Longitude,Canal,País,Fistel,Número da Estação,CNPJ,Coordenadas_do_Município,Fase,Situação,Unidade
44531,OM,AM-C7,RADIODIFUSAO RAINHA DO CEU LTDA,Bezerros,PE,57dbac68e0938,1590.0,C,-8.309444,-35.847500,NaN,BRA,50401708586,692535004,01889115000141,False,NaN,NaN,NaN
44532,OM,AM-C7,ALTERNATIVA FM LTDA,Lambari,MG,57dbac635271a,1590.0,C,-21.966667,-45.366667,NaN,BRA,50012009385,323663990,02406140000190,False,NaN,NaN,NaN
44533,OM,AM-C4,RADIO FLORESTA VERDE AM DE JOINVILLE LTDA,Joinville,SC,57dbac745cc94,1590.0,B,-26.275806,-48.780389,NaN,BRA,14030026656,323092799,79419263000190,False,NaN,NaN,NaN
44534,OM,AM-C0,NaN,Sertão,RS,57dbac72e2c41,1600.0,C,-27.983333,-52.250000,NaN,BRA,<NA>,NaN,<NA>,False,NaN,NaN,NaN
44535,OM,AM-C4,FUNDACAO METROPOLITANA PAULISTA,São Paulo,SP,57dbac7a245c7,1600.0,B,-23.501389,-46.691944,NaN,BRA,02032083310,9252657,50951847000120,False,NaN,NaN,NaN


In [ ]:
rd.loc[rd.Status.str.contains('AM'), 'Frequência'] = rd.loc[rd.Status.str.contains('AM'), 'Frequência'] / 1000 ; rd.loc[rd.Status.str.contains('AM'), 'Frequência'].head()

29283    0.54
29284    0.54
29285    0.54
29286    0.54
29287    0.54
Name: Frequência, dtype: float64

In [ ]:
rd = rd.fillna('-') ; rd.tail()

,Serviço,Status,Entidade,Município,UF,Id,Frequência,Classe,Latitude,Longitude,Canal,País,Fistel,Número da Estação,CNPJ,Coordenadas_do_Município,Fase,Situação,Unidade
44531,OM,AM-C7,RADIODIFUSAO RAINHA DO CEU LTDA,Bezerros,PE,57dbac68e0938,1.59,C,-8.309444,-35.847500,-,BRA,50401708586,692535004,01889115000141,False,-,-,-
44532,OM,AM-C7,ALTERNATIVA FM LTDA,Lambari,MG,57dbac635271a,1.59,C,-21.966667,-45.366667,-,BRA,50012009385,323663990,02406140000190,False,-,-,-
44533,OM,AM-C4,RADIO FLORESTA VERDE AM DE JOINVILLE LTDA,Joinville,SC,57dbac745cc94,1.59,B,-26.275806,-48.780389,-,BRA,14030026656,323092799,79419263000190,False,-,-,-
44534,OM,AM-C0,-,Sertão,RS,57dbac72e2c41,1.60,C,-27.983333,-52.250000,-,BRA,-,-,-,False,-,-,-
44535,OM,AM-C4,FUNDACAO METROPOLITANA PAULISTA,São Paulo,SP,57dbac7a245c7,1.60,B,-23.501389,-46.691944,-,BRA,02032083310,9252657,50951847000120,False,-,-,-


In [ ]:
rd.shape

(44536, 19)

In [ ]:
df = pd.DataFrame(columns=rd.columns)
for s in ['C1', 'C2', 'C3', 'C4', 'C7', 'RADCOM']:
    df = df.append(rd[rd.Status.str.contains(s)])
df = df.reset_index(drop=True); df.shape

(28028, 19)

In [ ]:
rd = df

In [ ]:
rd['Description'] = rd.Status + ', ' + rd.Classe + ', ' + rd.Entidade.str.title() + ' (' + \
                                rd.Fistel.astype('str') + ', ' + rd['Número da Estação'].astype('str') + '), ' + rd.Município + '/' + \
                                rd.UF

In [ ]:
rd.loc[rd.Coordenadas_do_Município == True, 'Description'] = rd.loc[rd.Coordenadas_do_Município == True, 'Description'] + '*'

In [ ]:
rd.head()

,Serviço,Status,Entidade,Município,UF,Id,Frequência,Classe,Latitude,Longitude,Canal,País,Fistel,Número da Estação,CNPJ,Coordenadas_do_Município,Fase,Situação,Unidade,Description
0,RTV,TV-C1,PREFEITURA MUNICIPAL DE JACARACI,Jacaraci,BA,57dbaae93fd68,57.0,C,-14.849720,-42.433330,2,BRA,50400457865,322641683,13677109000100,False,-,-,-,"TV-C1, C, Prefeitura Municipal De Jacaraci (50..."
1,RTV,TV-C1,PREFEITURA MUNICIPAL DE MONTE CARLO,Monte Carlo,SC,57dbab5752aa7,57.0,C,-27.222780,-50.979720,2,BRA,14030175570,323099955,95996104000104,False,-,-,-,"TV-C1, C, Prefeitura Municipal De Monte Carlo ..."
2,RTV,TV-C1,PREFEITURA MUNICIPAL DE PAPANDUVA,Papanduva,SC,57dbab57beb7c,57.0,C,-26.370280,-50.144440,2,BRA,14023497690,323089178,83102533000101,False,-,-,-,"TV-C1, C, Prefeitura Municipal De Papanduva (1..."
3,RTV,TV-C1,TV GAZETA DE ALAGOAS LTDA,Água Branca,AL,57dbaadc3eb7c,57.0,A,-9.266667,-37.933333,2,BRA,50400517868,323710034,12186524000106,False,-,-,-,"TV-C1, A, Tv Gazeta De Alagoas Ltda (504005178..."
4,RTV,TV-C1,PREFEITURA MUNICIPAL DE OEIRAS,Oeiras,PI,57dbab2f63994,57.0,B,-7.016667,-42.133333,2,BRA,19000170168,323231314,06553937000170,False,-,-,-,"TV-C1, B, Prefeitura Municipal De Oeiras (1900..."


In [ ]:
exp = rd[['Frequência', 'Latitude', 'Longitude', 'Description']].copy()
exp.columns = ['Frequency', 'Latitude', 'Longitude', 'Description']

In [ ]:
stel = read_stel('D:\OneDrive - ANATEL\GR01FI3\BaseDados').sort_values('Frequência').loc[:, TELECOM]

In [ ]:
stel.Serviço.unique()

array(['019', '035', '078', '079', '033', '108', '132', '507', '017',
       '011', '124', '125', '604', '064', '046', '053', '099', '012'],
      dtype=object)

In [ ]:
slma = (stel.Frequência >= 108) & (stel.Frequência <= 137)
telecom = stel[~slma].reset_index(drop=True)
slma = stel[slma].reset_index(drop=True)

In [ ]:
slma['Description'] = slma.Serviço + ', ' + slma.Entidade.str.title() + ' (' + slma.Fistel.astype('str') + ', ' +  slma['Número da Estação'].astype('str') + '), ' + \
                        slma['Município'] + '/' + slma.UF
telecom['Description'] = telecom.Serviço + ', ' + telecom.Entidade.str.title() + ' (' + telecom.Fistel.astype('str') + ', ' + telecom['Número da Estação'].astype('str') + '), ' + \
                           telecom['Município'] + '/' + telecom.UF

In [ ]:
slma = slma[['Frequência', 'Latitude', 'Longitude', 'Description']]
slma.columns = ['Frequency', 'Latitude', 'Longitude', 'Description']

In [ ]:
telecom = telecom[['Frequência', 'Latitude', 'Longitude', 'Description']]
telecom.columns = ['Frequency', 'Latitude', 'Longitude', 'Description']

In [ ]:
exp = exp.sort_values('Frequency')

In [ ]:
sub = {' Da ': ' da ', ' De ': ' de ', ' Do ': ' do ',
       ' Das ': ' das ', ' Dos ': ' dos ', ' E ': ' e '}

In [ ]:
for k,v in sub.items():
    exp['Description'] = exp.Description.str.replace(k,v)
    slma['Description'] = slma.Description.str.replace(k,v)
    telecom['Description'] = telecom.Description.str.replace(k,v)
    

In [ ]:
exp['Description'] = exp.Description.str.replace('\s{2,}', ' ', regex=True)
slma['Description'] = slma.Description.str.replace('\s{2,}', ' ', regex=True)
telecom['Description'] = telecom.Description.str.replace('\s{2,}', ' ', regex=True)    

In [ ]:
dest = r'D:\OneDrive - ANATEL\GR01FI3\BaseDados\FiltroAppAnalise'

for p in Path(dest).iterdir():
    p.unlink()

time = datetime.today().strftime("%Y%m%d_T%H%M%S")

with pd.ExcelWriter(f'{dest}/AnatelDB_{time}.xlsx') as workbook:
    exp.to_excel(workbook, sheet_name='Sheet1', index=False)
    slma.to_excel(workbook, sheet_name='Sheet2', index=False)
    telecom.to_excel(workbook, sheet_name='Sheet3', index=False)

In [ ]:
rd.groupby('Status').count()['Serviço'].sort_values(ascending=False)

Status
TV-C0        5789
TV-C3        4942
TV-C7        4738
RADCOM       4644
TV-C4        4081
FM-C0        3458
TV-C2        3251
FM-RES       2320
FM-C4        1534
TV-RES       1474
TV-C1        1206
AM-C0         875
FM-C3         875
FM-C7         609
AM-C7         604
FM-C2         535
TV-C5         427
FM-C1         400
TV-CV-04      336
FM-AT-03      281
AM-C3         265
AM-RES        232
FM-C5         208
AM-C4         165
TV-CP-03      153
TV-AT-03      132
FM-CI-01      125
TV-CP-01      109
AM-C2          98
AM-C1          81
FM-CP-03       77
AM-CE-01       76
FM-AT-01       52
TV-AT-01       52
TV-ATO-02      51
FM-CP-01       49
TV-CI-01       40
TV-EX-01       39
FM-ATO-02      34
FM-AT-02       25
FM-ATO-03      24
TV-C98         21
TV-AT-02       12
FM-EX-01       10
AM-CI-01        6
FM-CO-01        5
AM-CO-01        4
FM-C99          2
AM-CP-01        2
TV-CO-01        2
AM-C99          2
FM-EX-02        2
AM-C5           1
AM-AT-03        1
Name: Serviço, dtype:

In [ ]:
# from tqdm.notebook import tqdm
# geolocator = Nominatim(user_agent='anateldb')

# from geopy.extra.rate_limiter import RateLimiter
# geocode = RateLimiter(geolocator.reverse, min_delay_seconds=1)

# def info_from_coord(row):
#     loc = geocode((row.Latitude,row.Longitude))
#     if not hasattr(loc, 'raw'):
#         return row.País, row.País
#     if (address := loc.raw.get('address')):
#         if not (uf := address.get('state')): 
#             uf = row.País             
#         if not (cidade := address.get('city')):
#             if not (cidade := address.get('town')):
#                 cidade = row.País
#         return cidade, uf
#     return row.País, row.País